![milky](https://www.sciencealert.com/images/2019-10/processed/denis-degioanni-9wH624ALFQA-unsplash_1_1024.jpg)
# *Generating a MILKY WAY binary population*
## Goals and overview : *a Milky-Way-like fixed population*

Here, we plan to use `COSMIC`'s abilities to 

(1) come up with an initial ZAMS (Zero Age Main Sequence) population, 

(2) evolve it through a given binary evolution model again and again until 

(3) the output converges to a certain user-specified benchmark (or you hit the max number of iterations, whichever one happens first) to generate a representative Milky Way fixed population of doubly degenerate binaries.

Lucky for us, all those tasks are pretty much included in a single line of code whose output is the *fixed population*. But we'll spend some time exploring what exactly does that line need to run smoothly and what we can do with its outputs.

Galactic locations, adequate birth times and mass distribution consistent with a Milky Way model will be assigned in a second phase.








We start by installing the latest stable version, `cosmic 3.2.0` on the notebook.  *Note that the* `!` *is necessary when running* `cosmic` *on the notebook.*

In [ ]:
!pip install -q cosmic-popsynth==3.2.0

The fixed population is the output of the command `cosmic-pop`, which itself takes the following arguments, or inputs (the `-h` stands for help -- this is just displaying the most important details about the executable) :



In [ ]:
!cosmic-pop -h

usage: cosmic-pop [-h] --inifile INIFILE --final-kstar1 FINAL_KSTAR1
                  [FINAL_KSTAR1 ...] --final-kstar2 FINAL_KSTAR2
                  [FINAL_KSTAR2 ...] [--Niter NITER] [--Nstep NSTEP]
                  [-n NPROC] [--verbose]

optional arguments:
  -h, --help            show this help message and exit
  --inifile INIFILE     Name of ini file of params
  --final-kstar1 FINAL_KSTAR1 [FINAL_KSTAR1 ...]
                        Specify the final condition of kstar1 , you want
                        systems to end at for your samples
  --final-kstar2 FINAL_KSTAR2 [FINAL_KSTAR2 ...]
                        Specify the final condition of kstar2, you want
                        systems to end at for your samples
  --Niter NITER         Number of iterations of binaries to try, will check
                        ever Nstep for convergence
  --Nstep NSTEP         Number of binaries to try before checking for
                        convergence, it will check ever Nstep binaries

## Running `COSMIC`

You can find more info about the necessary input below, under the section **Some Additional Documentation**, but here's the main idea:


The parameters that `cosmic-pop` need to be specified to run are as follows:


*   `--inifile` 

The `Params.ini` file has been slightly modified to ask `cosmic` to output a `ThinDisk` population. The parts about **Binary Star Evolution** models have been left untouched.

You just need to have that file in the directory you're currently working in (I haven't checked if specifying a path works but this is Google Colab here so it's a bit different)

*   `--final_kstar1` and `--final_kstar2`

We want as output a population of white dwarf binaries, but we'll accept neutron stars and black holes. If we decide not to keep those, it will be easy to filter them out later, as we don't expect a large population of those anyway. Hence, the `kstar` values for both primary and secondary stars must be `10`, `11`, or `12` for white dwarfs, `13` for neutron stars or `14` for black holes.

The executable line will only accept single numbers (e.g.: `--kstar1 1`) or ranges (e.g.: `--kstar1 1 5`)

*   `--Nstep`, `--Niter` and `-n`

Now, the last part tells COSMIC to use 1 processor (`-n`) to evolve a maximum of 1e7 systems (`--Niter`) and check in every 5000 systems (`--Nstep`) to track how the shape of the distributions of the parameters specified in convergence-params change.

In [ ]:
!cosmic-pop --final-kstar1 10 14 --final-kstar2 10 14 --inifile Params.ini --Nstep 5000 --Niter 10000000 -n 1

Traceback (most recent call last):
  File "/usr/local/bin/cosmic-pop", line 103, in <module>
    BSEDict, seed_int, filters, convergence, sampling = utils.parse_inifile(args.inifile)
  File "/usr/local/lib/python3.6/dist-packages/cosmic/utils.py", line 1121, in parse_inifile
    BSEDict = dictionary['bse']
KeyError: 'bse'


Running `cosmic` will output:

*   a `log` text file that we can always check for details to see if anything went wrong
*   an `h5` file whose subsections can be extracted and read in the console as follows:

In [ ]:
import pandas as pd
import numpy as np

myfile = '/content/dat_ThinDisk_10_14_10_14.h5'

conv       = pd.read_hdf(myfile, key='conv')
total_mass = pd.read_hdf(myfile, key='mass_stars')
N_stars    = pd.read_hdf(myfile, key='n_stars')

## Scaling to an astrophysical population

> **Please note that the following uses an extensive amount of RAM, and is therefore impossible to execute on the notebook. It is still detailed here, as the code can be used as is on Quest, for example.**

Once the fixed population is simulated, you can scale the simulation to an astrophysical population by resampling the `conv` DataFrame to get a number of systems more in-line with expected Milky Way mass distributions.

In [ ]:
from cosmic import MC_samp

total_mass = max(np.array(total_mass))[0]
total_number_stars = max(np.array(N_stars))[0]

N_10_14_10_14_ThinDisk = MC_samp.mass_weighted_number(dat=conv, total_sampled_mass=total_mass, component_mass=MC_samp.select_component_mass('ThinDisk'))

thin_disk_sample = conv.sample(n=N_10_14_10_14_ThinDisk, replace=True)

The following lines of code serve to assign proper birth times and galactic coordinates to each system in our Thin Disk sample:

-    Age of the binary

What we start by doing is assigning uniformly random birthtimes `tbirth` anywhere from 0 to 10,000 Myr. Now, remember that `tphys` is the time it took each binary to reach the point in their evolutionary history that we recorded in our dataframe.

What we then do is filter out of our dataframe the systems for which `tbirth` + `tphys` surpasses the age of the Thin Disk itself, namely, an estimated 10,000 Myr.


-    Galactic location of the binary


In [ ]:
thin_disk_sample['tbirth'] = np.random.uniform(0, 10000, N_10_14_10_14_ThinDisk)
thin_disk_sample = thin_disk_sample.loc[thin_disk_sample.tbirth + thin_disk_sample.tphys < 10000]

xGx, yGx, zGx, inc, omega, OMEGA = MC_samp.galactic_positions(gx_component='ThinDisk', model='McMillan', size=len(thin_disk_sample))
thin_disk_sample['xGx']   = xGx
thin_disk_sample['yGx']   = yGx
thin_disk_sample['zGx']   = zGx
thin_disk_sample['inc']   = inc
thin_disk_sample['omega'] = omega
thin_disk_sample['OMEGA'] = OMEGA

thin_disk_sample

Now, here is just another formulation of what's above that is supposed to require less memory.
I'm dropping some extraneous columns in the dataframe I do not expect to use and go as far as to slice it into smaller, more manageable chunks.

In [ ]:
thin_disk_sample.drop(['evol_type', 'RROL_1', 'RROL_2', 'sep', 
                       'Vsys_1', 'Vsys_2', 'SNkick', 'SNtheta', 
                       'aj_1', 'aj_2', 'tms_1', 'tms_2', 
                       'massc_1', 'massc_2', 'rad_1', 'rad_2'], 
                        axis = 1, inplace = True)

thin_disk_sample['tbirth'] = np.random.uniform(0, 10000, N_10_14_10_14_ThinDisk)

thin_disk_sample = thin_disk_sample.loc[thin_disk_sample.tbirth + thin_disk_sample.tphys < 10000]

n = 100000  #chunk row size
mylist  = [thin_disk_sample[i:i+n] for i in range(0, thin_disk_sample.shape[0], n)]

for boo in range(0, thin_disk_sample.shape[0], n):
  mylist[boo] = mylist[boo].loc[mylist[boo].tbirth + mylist[boo].tphys < 10000]

thin_disk_sample = pd.concat(list_output)

thin_disk_sample.to_csv('thin_disk_WD_NS_BH.csv', index = False)

---
# Some additional documentation
## Inputs
### Some more info about `Params.ini`

The unavoidable input file -- it looks like a remnant of the legacy BSE code upon which `cosmic` is built. The existing `cosmic` documentation provides one such file that can be used as is -- but modifying it yourself in your favorite text editor isn't too difficult.

[What's the `.ini` format?](https://searchwindowsserver.techtarget.com/definition/INI)

What exactly the file contains in detailed on the `cosmic` [documentation website](https://cosmic-popsynth.github.io/docs/stable/inifile/index.html) quite extensively, but for completeness sake, I'll provide a brief overview.


#### filters
*    What exactly should I retain in the final output files? Do I want only the final state of each binary, or should I keep space for their states mid-evolution? 
*    Also, do I only want to keep binaries who at the end of the evolution time are still untouched (albeit maybe remnants now) or am I fine with keeping some that have merged or have been disrupted in some way?

#### sampling
*    What models am I using to generate the initial Zero Age Main Sequence population?
*    What galaxy component am I interested in sampling? (each have different star formation history)
*    Metallicity

#### convergence parameters
*    What are the parameters I want to see converging? Mass, eccentricity, orbital period, etc ... ?
*    At what evolutionary stage are we checking for that convergence?
*    Should we filter out from our `bcm`, `bpp` and `initCond` dataframe the binaries that didn't make the cut into our `conv` dataframe?
*    What's my convergence tolerance? How close do the parameters have to be for me to accept them?

#### random seed
*    for reproducibility. Note that for each iteration inside `cosmic-pop` (meaning, everytime we randomly sample a new ZAMS population), we add `Nstep` to the `numpy.random.seed` used. This is so that each iteration gets a unique, determinable seed.

#### BSE 
*    Sampling flags (set length of time steps for the `bcm` dataframe)
*    Wind flags (choose model for wind mass loss & set additional wind parameters)
*    Common enveloppe flags
*    Kick flags
*    Remnant mass flags
*    remnant spin flags
*    Mass transfer flags
*    Tides flags
*    White dwarf flags
*    Pulsar flags
*    Mixing variables
*    Magnetic braking flags
*    Convective vs. radiative boundaries


### Selecting White Dwarves only
The `kstar` value in the output dataframes specifies the evolutionary state of the star.
Namely:

`kstar` | evolutionary state
---     | ---
`0`     | Main Sequence < 0.7 ${\mathrm{M}_\odot}$
`1`     | Main Sequence > 0.7 ${\mathrm{M}_\odot}$
...     | ...
`10`    | Helium White Dwarf
`11`    | Carbon/Oxygen White Dwarf
`12`    | Oxygen/Neon White Dwarf
`13`    | Neutron Star
`14`    | Black Hole
...     | ...


### Understanding the outputs
The `cosmic` outputs are in `pandas.DataFrame` format, a highly polyvalent python object. 
Variables with the subscript `_1` are referring to the primary star. Conversely, a `_2` ending indicates it is referring to the secondary star.

Here are the some of the main outputs you might find laying around

#### [`BPP` dataframe](https://cosmic-popsynth.github.io/docs/stable/output_info/index.html#bpp-dataframe)

The `BPP` dataframe records a selection of binary parameters at key evolutionary changes, such as `kstar` change, contact, binary disruption or supernova of either primary or secondary.

It is the evolutionary history of each binary.

parameter   | Name                       |    Units  ..... 
---         | ---                        |  ---
`tphys`     | Evolution time             | $[{\rm{Myr}}]$
`mass_1`    | Primary mass               | $[{\mathrm{M}_\odot}]$
`kstar_1`   | Primary evolutionary state |
`sep`       | Semimajor axis             | $[{\mathrm{R}_\odot}]$
`porb`      | Orbital period                              | $[{\rm{days}}]$
`RROL_1`    | Primary radius divided by Roche lobe radius |
`evol_type` | Type of key moments in evolution            |
`Vsys_1`    | Change in systemic velocity due to first SN | $[{\rm{km/s}}]$
`SNkick`    | Magnitude of SN natal kick                  | $[{\rm{km/s}}]$
`SNtheta`   | Angular change in orbital plane due to SN   | $[{\rm{degrees}}]$
`aj_1`      | Effective age of the primary                | $[{\rm{Myr}}]$
`tms_1`     | Primary main sequence lifetime              | $[{\rm{Myr}}]$
`massc_1`   | Primary core mass                           | $[{\mathrm{M}_\odot}]$
`rad_1`     | Primary radius                              | $[{\mathrm{R}_\odot}]$
`bin_num`   | Unique binary index that is consistent 
            | across initial conditions,
            | BCM and BPP DataFrames

#### [`BCM` dataframe](https://cosmic-popsynth.github.io/docs/stable/output_info/index.html#bcm-dataframe)

The `BCM` dataframe records a slightly different selection of binary parameters at user-specified timesteps in the evolution.

It is (by default) the final state of each binary.


parameter       | Name                          |    Units  .....                    |
---             | ---                           |  ---                               | 
`tphys`         | Evolution time                | $[{\rm{Myr}}]$                     |
`kstar_1`       | Evolutionary state            |
`mass0_1`       | Previous evolutionary stage primary mass | $[{\mathrm{M}_\odot}]$
`mass_1`        | Primary mass                  | $[{\mathrm{M}_\odot}]$
`lumin_1`       | Primary luminosity            | $[{\mathrm{L}_\odot}]$
`rad_1`         | Primary radius                | $[{\mathrm{R}_\odot}]$
`teff_1`        | Primary effective temperature | $[{\rm{K}}]$
`massc_1`       | Primary core mass             | $[{\mathrm{M}_\odot}]$
`radc_1`        | Primary core radius           | $[{\mathrm{R}_\odot}]$
`menv_1`        | Primary envelope mass         | $[{\mathrm{M}_\odot}]$
`renv_1`        | Primary envelope radius       | $[{\mathrm{R}_\odot}]$
`epoch_1`       | Primary epoch                 | $[\rm{Myr}]$
`ospin_1`       | Primary spin                  | $[\rm{rad/yr}]$
`deltam_1`      | Primary mass transfer rate    | $[{\mathrm{M}_\odot/\rm{yr}}]$
`RROL_1`        | Primary radius divided by Roche lobe radius
`porb`          | Orbital period                | $[\rm{days}]$
`sep`           | Semimajor axis                | $[\mathrm{R}_{\odot}]$
`ecc`           | Eccentricity
`B_0_1`         | Initial neutron star magnetic field | $[{\rm{G}}]$
`SNkick_1`      | Magnitude of first natal kick       | $[{\rm{km/s}}]$
`Vsys_final`    | Final systemic velocity magnitude   | $[{\rm{km/s}}]$
`SNtheta_final` | Final systemic velocity angle       | $[{\rm{degrees}}]$
`SN_1`          | Supernova type:
                | `1` : Fe Core-collapse SN
                | `2` : Electron capture SN
                | `3` : Ultra-stripped supernovae
                | `4` : Accretion induced collapse SN
                | `5` : Merger induced collapse
                | `6` : Pulsational-pair instability
                | `7` : Pair instability SN
`bin_state`     | State of the binary: 
                | `0` : binary
                | `1` : merged
                | `2` : disrupted
`merger_type`   | String of the kstar's in the merger, 
                | `-001` if not merged
`bin_num`       | Unique binary index that is consistent 
                | across initial conditions,
                | BCM and BPP DataFrames


---
# Additional lines of code for doing things

In [ ]:
total_number_stars

3665457